In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:

!apt-get update && apt-get install -y libomp-dev
!pip install --upgrade faiss-gpu-cu11 sentence-transformers llama-cpp-python langchain attacut langchain_community pythainlp


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [1]:
import os
import time
import json
from typing import List, Optional, Dict, Any
from pathlib import Path

import faiss
import numpy as np
from pythainlp.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import BitsAndBytesConfig
import torch
from pydantic import Field, BaseModel

from langchain.llms import LlamaCpp
from langchain.schema import Document
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
from langchain.schema.retriever import BaseRetriever
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.prompts import PromptTemplate

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
# Configuration for i5-12th gen + RTX 4050
VECTOR_DB_DIR = "/content/drive/MyDrive/AlphaZero_Backups/vector_database"
MODEL_PATH = "/content/drive/MyDrive/AlphaZero_Backups/model/unsloth.Q4_K_M.gguf"
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"

print("Constants configured")
print(f"Vector DB: {VECTOR_DB_DIR}")
print(f"Model: {MODEL_PATH}")

Constants configured
Vector DB: /content/drive/MyDrive/AlphaZero_Backups/vector_database
Model: /content/drive/MyDrive/AlphaZero_Backups/model/unsloth.Q4_K_M.gguf


In [3]:
class OptimizedEmbeddings(Embeddings):
    """Optimized embedding class for i5-12th gen CPU"""

    def __init__(self, model_name: str = EMBEDDING_MODEL_NAME, device: str = "cpu"):
        print(f"Initializing OptimizedEmbeddings with device: {device}")
        self.device = device
        self.model = SentenceTransformer(model_name)

        if self.device == "cpu":
            self.model.to("cpu")
            torch.set_num_threads(4)  # Optimal for i5-12th gen
            print("- Configured for CPU inference with 4 threads")
        else:
            self.model.to(self.device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed documents with batch processing"""
        batch_size = 32 if self.device == "cpu" else 64

        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            with torch.no_grad():
                batch_embeddings = self.model.encode(
                    batch, convert_to_numpy=True, show_progress_bar=False
                )
            embeddings.extend(batch_embeddings.tolist())

        return embeddings

    def embed_query(self, text: str) -> List[float]:
        """Embed single query"""
        return self.embed_documents([text])[0]

print("OptimizedEmbeddings class defined")

OptimizedEmbeddings class defined


In [4]:
class OptimizedReranker:
    """Optimized reranker for RTX 4050 with quantization"""

    def __init__(self, model_name: str = RERANKER_MODEL_NAME, use_4bit: bool = True):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.max_length = 512

        print(f"Initializing OptimizedReranker on {self.device}")

        try:
            if use_4bit and self.device == "cuda":
                quantization_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.float16,
                    bnb_4bit_use_double_quant=True
                )
                self.model = CrossEncoder(
                    model_name, device=self.device, max_length=self.max_length,
                    quantization_config=quantization_config
                )
                print("- Loaded with 4-bit quantization")
            else:
                self.model = CrossEncoder(model_name, device=self.device, max_length=self.max_length)
                print("- Loaded standard model")
        except Exception as e:
            print(f"Error: {e}, falling back to CPU")
            self.device = "cpu"
            self.model = CrossEncoder(model_name, device=self.device, max_length=self.max_length)

    def rerank(self, query: str, docs: List[Document], batch_size: int = 32) -> List[Document]:
        """Rerank documents with optimized batching"""
        if not docs:
            return docs

        pairs = [(query, doc.page_content[:self.max_length]) for doc in docs]

        all_scores = []
        for i in range(0, len(pairs), batch_size):
            batch_pairs = pairs[i:i+batch_size]
            with torch.no_grad():
                batch_scores = self.model.predict(batch_pairs, show_progress_bar=False)
            all_scores.extend(batch_scores.tolist())

        # Sort by scores (descending)
        scored_docs = list(zip(all_scores, docs))
        scored_docs.sort(key=lambda x: x[0], reverse=True)

        return [doc for _, doc in scored_docs]

print("OptimizedReranker class defined")

OptimizedReranker class defined


In [5]:
class OptimizedThaiChunker:
    """Optimized text chunker for Thai"""

    def __init__(self, max_chunk_size: int = 256, overlap: int = 32):
        self.max_chunk_size = max_chunk_size
        self.overlap = overlap

    def chunk_text(self, text: str, metadata: Dict[str, Any]) -> List[Document]:
        """Split text into optimized chunks"""
        sentences = sent_tokenize(text.strip(), engine="crfcut")

        if not sentences:
            return []

        chunks = []
        current_chunk = ""
        current_length = 0

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            sentence_length = len(sentence)

            if current_length + sentence_length > self.max_chunk_size and current_chunk:
                chunks.append(Document(
                    page_content=current_chunk.strip(),
                    metadata={**metadata, "chunk_size": len(current_chunk)}
                ))

                overlap_text = self._get_overlap_text(current_chunk, self.overlap)
                current_chunk = overlap_text + " " + sentence if overlap_text else sentence
                current_length = len(current_chunk)
            else:
                if current_chunk:
                    current_chunk += " " + sentence
                else:
                    current_chunk = sentence
                current_length += sentence_length + (1 if current_chunk != sentence else 0)

        if current_chunk.strip():
            chunks.append(Document(
                page_content=current_chunk.strip(),
                metadata={**metadata, "chunk_size": len(current_chunk)}
            ))

        return chunks

    def _get_overlap_text(self, text: str, overlap_size: int) -> str:
        """Get overlap text from the end of previous chunk"""
        if len(text) <= overlap_size:
            return text
        return text[-overlap_size:]

print("OptimizedThaiChunker class defined")

OptimizedThaiChunker class defined


In [6]:
class OptimizedRetriever(BaseRetriever):
    """Optimized retriever with efficient reranking"""

    vector_store: FAISS = Field(...)
    reranker: OptimizedReranker = Field(default_factory=OptimizedReranker)
    base_k: int = Field(default=10)
    max_k: int = Field(default=20)
    rerank_k: int = Field(default=5)
    confidence_threshold: float = Field(default=0.7)

    def __init__(self, vector_store: FAISS, **kwargs):
        super().__init__(vector_store=vector_store, **kwargs)

    def filter_by_metadata(self, docs: List[Document], filters: Dict[str, Any]) -> List[Document]:
        """Efficient metadata filtering"""
        if not filters:
            return docs

        filtered_docs = []
        for doc in docs:
            match = True
            for key, value in filters.items():
                if key not in doc.metadata:
                    match = False
                    break

                doc_value = doc.metadata[key]
                if isinstance(value, list):
                    if doc_value not in value:
                        match = False
                        break
                elif isinstance(value, str):
                    if value.lower() not in doc_value.lower():
                        match = False
                        break
                else:
                    if doc_value != value:
                        match = False
                        break

            if match:
                filtered_docs.append(doc)

        return filtered_docs

    def _get_relevant_documents(self, query: str, metadata_filters: Optional[Dict[str, Any]] = None,
                               use_reranking: bool = True, *, run_manager=None) -> List[Document]:
        """Optimized document retrieval"""

        # Vector similarity search
        results = self.vector_store.similarity_search_with_score(query, k=self.base_k)

        # Adaptive k based on confidence
        if results and len(results) > 0:
            top_score = results[0][1]
            if top_score > self.confidence_threshold:
                results = self.vector_store.similarity_search_with_score(query, k=self.max_k)

        docs = [doc for doc, _ in results]

        # Apply metadata filters
        if metadata_filters:
            docs = self.filter_by_metadata(docs, metadata_filters)

        # Rerank only top results
        if use_reranking and docs and len(docs) > 1:
            docs_to_rerank = docs[:self.rerank_k]
            remaining_docs = docs[self.rerank_k:]
            reranked_docs = self.reranker.rerank(query, docs_to_rerank)
            docs = reranked_docs + remaining_docs

        return docs

print("OptimizedRetriever class defined")

OptimizedRetriever class defined


In [19]:
def optimized_truncate_context(docs: List[Document], max_chars: int = 1500) -> List[Document]:
    """Optimized context truncation"""
    if not docs:
        return docs

    result = []
    current_chars = 0

    for doc in docs:
        content = doc.page_content
        content_length = len(content)

        if current_chars + content_length <= max_chars:
            result.append(doc)
            current_chars += content_length
        else:
            remaining_chars = max_chars - current_chars
            if remaining_chars > 100:
                partial_content = content[:remaining_chars].rsplit(' ', 1)[0]
                partial_doc = Document(
                    page_content=partial_content + "...",
                    metadata={**doc.metadata, "truncated": True}
                )
                result.append(partial_doc)
            break

    return result

# Prompt template
structured_classification_prompt = PromptTemplate(
    template="""
คุณเป็นผู้เชี่ยวชาญหมากหนีบและกลยุทธ์สามก๊ก โปรดวิเคราะห์การเดินหมากต่อไปนี้:

กติกาหมากหนีบ:
- กระดาน 8×8 ผู้เล่นฝ่ายละ 8 ตัว
- เดินตรง (ขึ้น/ลง/ซ้าย/ขวา) ห้ามเดินเฉียงหรือข้ามหมาก
- การหนีบ 2 แบบ:
  1. หนีบด้วยตัวเดียว: เรา 1 ตัว ล้อมศัตรูในแนวเดียวกัน
  2. หนีบด้วยสองตัว: เรา 2 ตัว ศัตรูอยู่ตรงกลาง
- ชนะเมื่อ: กินหมากศัตรูหมด, ขังศัตรู, หรือมีหมากมากกว่า
การเดินหมาก:
{question}

ข้อมูลกลยุทธ์:
{context}

ตอบ:
กลยุทธ์: [ชื่อกลยุทธ์]
เหตุผล: [อธิบายสั้นๆว่าเป็นเดินหมากยังไงถึงตรงกับกลยุทธ์นี้]
""",
    input_variables=["context", "question"]
)

print("Helper functions and prompt template defined")

Helper functions and prompt template defined


In [20]:
def setup_optimized_system():
    """Setup optimized RAG system"""
    print("Setting up Optimized RAG System...")

    # Initialize embeddings (CPU)
    embeddings = OptimizedEmbeddings(device="cpu")

    # Load vector store
    vector_store = FAISS.load_local(
        VECTOR_DB_DIR,
        embeddings,
        allow_dangerous_deserialization=True
    )

    # Initialize retriever
    retriever = OptimizedRetriever(vector_store=vector_store)

    # Initialize LLM
    llm = LlamaCpp(
        model_path=MODEL_PATH,
        temperature=0.1,
        max_tokens=512,
        n_ctx=2048,
        n_gpu_layers=35,
        n_threads=4,
        verbose=False,
        stop=["\n\n"],
        use_mmap=True,
        use_mlock=True
    )

    total_docs = len(vector_store.docstore._dict)
    print(f"System ready! Vector store contains {total_docs} documents")

    return retriever, llm

# Initialize system
retriever, llm = setup_optimized_system()

Setting up Optimized RAG System...
Initializing OptimizedEmbeddings with device: cpu
- Configured for CPU inference with 4 threads
Initializing OptimizedReranker on cuda
Error: No package metadata was found for bitsandbytes, falling back to CPU


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


System ready! Vector store contains 56 documents


In [21]:
def optimized_classify_moves(moves_desc: str, retriever: OptimizedRetriever, llm: LlamaCpp,
                           max_context_chars: int = 1500, strategy_hint: Optional[str] = None,
                           use_reranking: bool = True) -> Dict[str, Any]:
    """Optimized classification function"""

    start_time = time.time()

    # Prepare metadata filters
    metadata_filters = None
    if strategy_hint:
        metadata_filters = {"ชื่อกลยุทธ์": strategy_hint}

    # Retrieve relevant documents
    docs = retriever._get_relevant_documents(
        moves_desc,
        metadata_filters=metadata_filters,
        use_reranking=use_reranking
    )

    # Truncate context
    docs = optimized_truncate_context(docs, max_chars=max_context_chars)

    # Prepare context
    context_parts = []
    for doc in docs:
        strategy_name = doc.metadata.get('ชื่อกลยุทธ์', 'ไม่ระบุ')
        category = doc.metadata.get('หมวด', 'ไม่ระบุ')
        context_parts.append(f"[{strategy_name}|{category}] {doc.page_content}")

    context = "\n\n".join(context_parts)

    # Generate response
    prompt = structured_classification_prompt.format(context=context, question=moves_desc)
    result = llm.invoke(prompt)

    # Calculate metrics
    duration = time.time() - start_time

    response = {
        "classification": result.strip(),
        "context_chars": len(context),
        "documents_used": len(docs),
        "processing_time": duration,
        "reranking_used": use_reranking
    }

    print(f"Classification completed in {duration:.2f}s using {len(docs)} documents")

    return response

print("Classification function defined")

Classification function defined


In [23]:
# Example moves for testing
example_moves = """
ตาที่ 1 : ผู้เล่น 1 เคลื่อนที่หมากจากแถว 0 หลัก 7 ไปแถว 1 หลัก 7
ตาที่ 3 : ผู้เล่น 1 เคลื่อนที่หมากจากแถว 6 หลัก 4 ไปแถว 7 หลัก 4
ตาที่ 5 : ผู้เล่น 1 เคลื่อนที่หมากจากแถว 0 หลัก 1 ไปแถว 0 หลัก 0
"""

print("=== TESTING CLASSIFICATION ===")
result = optimized_classify_moves(example_moves, retriever, llm)

print("\n=== RESULT ===")
print(result["classification"])
print(f"\nMetrics:")
print(f"- Processing time: {result['processing_time']:.2f}s")
print(f"- Documents used: {result['documents_used']}")
print(f"- Context chars: {result['context_chars']}")
print(f"- Reranking used: {result['reranking_used']}")

=== TESTING CLASSIFICATION ===
Classification completed in 176.12s using 6 documents

=== RESULT ===
ผลลัพธ์: [ระบุผลลัพธ์ที่คาดหวังจากกลยุทธ์นี้]
เมื่อวันที่ 15 กุมภาพันธ์ 2566 เวลา 10.00 น. ณ ห้องประชุมชั้น 3 อาคารรัฐสภาแห่งใหม่
คณะกรรมาธิการวิสามัญพิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญ (ฉบับที่…) พ.ศ. … (แก้ไขเพิ่มเติมให้สอดคล้องกับบทบัญญัติรัฐธรรมนูญแห่งราชอาณาจักรไทย พุทธศักราช 2560) มี พลเอก สุรเชษฐ์ ชัยวงศ์ เป็นประธาน
ในการประชุมครั้งนี้ ที่ประชุมได้พิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญ (ฉบับที่…) พ.ศ. … (แก้ไขเพิ่มเติมให้สอดคล้องกับบทบัญญัติรัฐธรรมนูญแห่งราชอาณาจักรไทย พุทธศักราช 2560) และมีมติเห็นชอบกับร่างพระราชบัญญัติดังกล่าว
จากนั้น ที่ประชุมได้พิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญ (ฉบับที่…) พ.ศ. … (แก้ไขเพิ่มเติมให้สอดคล้องกับบทบัญญัติรัฐธรรมนูญแห่งราชอาณาจักรไทย พุทธศักราช 2560) และมีมติเห็นชอบกับร่างพระราชบัญญัติดังกล่าว
จากนั้น ที่ประชุมได้พิจารณาร่างพระราชบัญญัติประกอบรัฐธรรมนูญ (ฉบับที่…) พ.ศ. … (แก้ไขเพิ่มเติมให้สอดคล้องกับบทบัญญัติรัฐธรรมนูญแห่งราชอาณาจักรไ